In [2]:
import numpy as np
from datasets import load_dataset
import tiktoken
import multiprocessing

import warnings
warnings.filterwarnings("ignore", category=UserWarning,
                        message=r"^\nThe secret `HF_TOKEN` does not exist.*")

tokenizer = tiktoken.get_encoding("gpt2")
eot = tokenizer._special_tokens['<|endoftext|>']
dataset = load_dataset("HuggingFaceFW/fineweb-edu",
                           name="CC-MAIN-2024-10", split="train", streaming=True)
pool = multiprocessing.Pool(1)



In [7]:
def worker(row):
    return tokenizer.encode(row['text'])

def gen_token_chunks(n_token: int, n_processes: int = None):
    tokens = []
    for row_tokens in pool.imap(worker, dataset):
        tokens.append(eot)
        tokens.extend(row_tokens)
        if len(tokens) > n_token:
            chunk, reminder = tokens[:n_token], tokens[n_token:]
            tokens = reminder
            yield np.array(chunk, dtype=np.uint16)
    yield np.array(tokens, dtype=np.uint16)  # the last chunk

for i in gen_token_chunks(1000):
  break

KeyboardInterrupt: 

: 

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (1000,) + inhomogeneous part.